### This notebook explores the Python API client for Federal Reserve Economic Data (FRED).

# Import libraries and load configurations

In [8]:
import os
import pyfredapi as pf
import numpy as np
import pandas as pd

from dotenv import load_dotenv

In [14]:
load_dotenv(r"C:\Users\mushj\OneDrive\Desktop\WORK\Financial Analytics\.env")

True

In [15]:
API_KEY = os.getenv('FRED_API_KEY')

# View single series

In [16]:
# get series info
series_info = pf.get_series_info(series_id="GDP", api_key=API_KEY)
print(*series_info, sep="\n")

('id', 'GDP')
('realtime_start', '2024-12-19')
('realtime_end', '2024-12-19')
('title', 'Gross Domestic Product')
('observation_start', '1947-01-01')
('observation_end', '2024-07-01')
('frequency', 'Quarterly')
('frequency_short', 'Q')
('units', 'Billions of Dollars')
('units_short', 'Bil. of $')
('seasonal_adjustment', 'Seasonally Adjusted Annual Rate')
('seasonal_adjustment_short', 'SAAR')
('last_updated', '2024-12-19 07:55:01-06')
('popularity', 93)
('notes', 'BEA Account Code: A191RC\n\nGross domestic product (GDP), the featured measure of U.S. output, is the market value of the goods and services produced by labor and property located in the United States.For more information, see the Guide to the National Income and Product Accounts of the United States (NIPA) and the Bureau of Economic Analysis (http://www.bea.gov/national/pdf/nipaguid.pdf).')


In [10]:
# get category of the series
cat_info = pf.get_series_categories(series_id=series_info.id, api_key=API_KEY)
cat_info

{'categories': [{'id': 106, 'name': 'GDP/GNP', 'parent_id': 18}]}

# Explore category

In [17]:
cat_id = cat_info["categories"][0]["id"]
cat_id

106

In [18]:
# get the SeriesInfo of all series in the category
all_series_in_cat = pf.get_category_series(category_id=cat_id, api_key=API_KEY)
print(len(all_series_in_cat))

377


In [19]:
# get the popularity value for all series
popularities = [series_info.popularity for series_info in all_series_in_cat.values()]

# get frequency for all series
frequencies = [series_info.frequency_short for series_info in all_series_in_cat.values()]

In [20]:
# examine distribution of popularity scores
pd.DataFrame(popularities).describe(percentiles=np.arange(0.6,1,0.05))

,0
count,377.000000
mean,7.535809
std,15.828180
min,0.000000
50%,1.000000
60%,1.000000
65%,2.000000
70%,3.000000
75%,4.000000
80%,7.000000


In [76]:
# examine counts of Quarterly vs. Annual frequencies
pd.DataFrame(frequencies).value_counts()

Q    615
A    161
dtype: int64

In [15]:
# print details of popular series in category
IDs = []

for series_info in all_series_in_cat.values():
    if series_info.popularity > 20:
        print(series_info.title, series_info.id, 
              series_info.frequency_short,
              series_info.seasonal_adjustment_short,
              sep=" | ")
        IDs.append(series_info.id)

Real Gross Domestic Product | A191RL1A225NBEA | A | NSA
Real Gross Domestic Product | A191RL1Q225SBEA | Q | SAAR
Real Gross Domestic Product | A191RO1Q156NBEA | Q | SA
Gross Domestic Product | A191RP1A027NBEA | A | NSA
Gross Domestic Product | A191RP1Q027SBEA | Q | SAAR
Real gross domestic income | A261RX1Q020SBEA | Q | SAAR
Nonfinancial corporate business: Profits before tax (without IVA and CCAdj) | A464RC1Q027SBEA | Q | SAAR
Gross domestic product per capita | A939RC0A052NBEA | A | NSA
Gross domestic product per capita | A939RC0Q052SBEA | Q | SAAR
Real gross domestic product per capita | A939RX0Q048SBEA | Q | SAAR
Brave-Butters-Kelley Real Gross Domestic Product | BBKMGDP | M | SA
Brave-Butters-Kelley Leading Index | BBKMLEIX | M | SA
Change in Private Inventories | CBI | Q | SAAR
Gross Domestic Product | GDP | Q | SAAR
Gross Domestic Product | GDPA | A | NSA
Real Gross Domestic Product | GDPC1 | Q | SAAR
FOMC Summary of Economic Projections for the Growth Rate of Real Gross Domesti

# Test API call

In [18]:
params = {"observation_start": "2000-01-01"}

In [19]:
ID = "VAPGDPAER"
df = pf.get_series(series_id=ID, api_key=API_KEY, **params)
print(df.shape)

(73, 4)


In [20]:
# get series info
series_info = pf.get_series_info(series_id=ID, api_key=API_KEY)
print(*series_info, sep="\n")

('id', 'VAPGDPAER')
('realtime_start', '2023-10-21')
('realtime_end', '2023-10-21')
('title', 'Value Added by Industry: Arts, Entertainment, Recreation, Accommodation, and Food Services: Arts, Entertainment, and Recreation as a Percentage of GDP')
('observation_start', '2005-01-01')
('observation_end', '2023-01-01')
('frequency', 'Quarterly')
('frequency_short', 'Q')
('units', 'Percent')
('units_short', '%')
('seasonal_adjustment', 'Not Seasonally Adjusted')
('seasonal_adjustment_short', 'NSA')
('last_updated', '2023-06-29 11:04:40-05')
('popularity', 1)
('notes', 'Value added represents the sum of the costs-incurred and the incomes-earned in production, and consists of compensation of employees, taxes on production and imports, less subsidies, and gross operating surplus.')


In [21]:
df

,realtime_start,realtime_end,date,value
0,2023-10-21,2023-10-21,2005-01-01,0.9
1,2023-10-21,2023-10-21,2005-04-01,0.9
2,2023-10-21,2023-10-21,2005-07-01,0.9
3,2023-10-21,2023-10-21,2005-10-01,0.9
4,2023-10-21,2023-10-21,2006-01-01,0.9
...,...,...,...,...
68,2023-10-21,2023-10-21,2022-01-01,1.0
69,2023-10-21,2023-10-21,2022-04-01,1.0
70,2023-10-21,2023-10-21,2022-07-01,1.1
71,2023-10-21,2023-10-21,2022-10-01,1.1
